## Book recomendatoin using Cluster | hybrid Recomandation

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

In [3]:
os.chdir("..")

In [10]:
df = pd.read_csv("data/BX-Books.csv", sep=';', on_bad_lines='skip', encoding="latin-1" )

/var/folders/7r/vsv5qx453z3931l81y_t76y80000gn/T/ipykernel_23797/287028551.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("data/BX-Books.csv", sep=';', on_bad_lines='skip', encoding="latin-1" )


In [11]:
df.head()

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...


In [12]:
df.shape

(271360, 8)

In [13]:
df.columns

Index(['ISBN', 'Book-Title', 'Book-Author', 'Year-Of-Publication', 'Publisher',
       'Image-URL-S', 'Image-URL-M', 'Image-URL-L'],
      dtype='object')

In [14]:
df = df[['ISBN','Book-Title', 'Book-Author', 'Year-Of-Publication', 'Publisher',
       'Image-URL-L']]

In [15]:
df.head()

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...


In [18]:
df = df.rename(
    columns={
        "Book-Title": "title",
        "Book-Author": "author",
        "Year-Of-Publication": "year",
        "Publisher": "publisher",
        "Image-URL-L": "image"
    }
)

In [19]:
df.columns

Index(['ISBN', 'title', 'author', 'year', 'publisher', 'image'], dtype='object')

In [21]:
df_user = pd.read_csv("data/BX-Users.csv", on_bad_lines="skip", sep=";", encoding="latin-1")

In [22]:
df_user.head()

,User-ID,Location,Age
0,1,"nyc, new york, usa",NaN
1,2,"stockton, california, usa",18.0
2,3,"moscow, yukon territory, russia",NaN
3,4,"porto, v.n.gaia, portugal",17.0
4,5,"farnborough, hants, united kingdom",NaN


In [31]:
df_user = df_user.rename(columns={
    "User-ID": "user_id",
    "Age": "age"
})

In [33]:
df_user.columns

Index(['user_id', 'Location', 'age'], dtype='object')

In [23]:
df_user.shape

(278858, 3)

In [24]:
df_ratings = pd.read_csv("data/BX-Book-Ratings.csv", sep=";", on_bad_lines="skip", encoding="latin-1")

In [25]:
df_ratings.head()

,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6


In [28]:
df_ratings = df_ratings.rename(columns={
    "User-ID": "user_id",
    "Book-Rating": "rating"
})

In [29]:
df_ratings.columns

Index(['user_id', 'ISBN', 'rating'], dtype='object')

In [26]:
df_ratings.shape

(1149780, 3)

In [27]:
print(df.shape)
print(df_user.shape)
print(df_ratings.shape)

(271360, 6)
(278858, 3)
(1149780, 3)


In [38]:
df_ratings.user_id.value_counts()

user_id
11676     13602
198711     7550
153662     6109
98391      5891
35859      5850
          ...  
116180        1
116166        1
116154        1
116137        1
276723        1
Name: count, Length: 105283, dtype: int64

In [79]:
x = df_ratings.user_id.value_counts() < 200 #removing outliers

In [80]:
x[x].shape # df such that the conditoin above is true

(104378,)

In [81]:
indexes = x[x].index

In [82]:
df_ratings = df_ratings[df_ratings.user_id.isin(indexes)]

In [83]:
df_ratings.shape

(622224, 3)

In [85]:
print(df_ratings.columns)
print(df.columns)

Index(['user_id', 'ISBN', 'rating'], dtype='object')
Index(['ISBN', 'title', 'author', 'year', 'publisher', 'image'], dtype='object')


In [86]:
df_ratings_with_book = df_ratings.merge(df, on="ISBN") ## INNER JOIN

In [91]:
df_ratings.shape

(622224, 3)

In [87]:
df_ratings_with_book.head(20)

,user_id,ISBN,rating,title,author,year,publisher,image
0,276725,034545104X,0,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,http://images.amazon.com/images/P/034545104X.0...
1,2313,034545104X,5,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,http://images.amazon.com/images/P/034545104X.0...
2,8680,034545104X,5,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,http://images.amazon.com/images/P/034545104X.0...
3,10314,034545104X,9,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,http://images.amazon.com/images/P/034545104X.0...
4,28266,034545104X,0,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,http://images.amazon.com/images/P/034545104X.0...
5,39002,034545104X,0,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,http://images.amazon.com/images/P/034545104X.0...
6,50403,034545104X,9,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,http://images.amazon.com/images/P/034545104X.0...
7,56157,034545104X,0,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,http://images.amazon.com/images/P/034545104X.0...
8,59102,034545104X,0,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,http://images.amazon.com/images/P/034545104X.0...
9,59287,034545104X,0,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,http://images.amazon.com/images/P/034545104X.0...


In [90]:
df_ratings_with_book.title.isnull().sum()

0

In [104]:
df_num_rating = df_ratings_with_book.groupby('title').rating.count().reset_index()

In [105]:
df_num_rating.shape

(156592, 2)

In [109]:
df_num_rating.head()

,title,rating
0,A Light in the Storm: The Civil War Diary of ...,2
1,"Ask Lily (Young Women of Faith: Lily Series, ...",1
2,Deceived,1
3,Earth Prayers From around the World: 365 Pray...,7
4,Final Fantasy Anthology: Official Strategy Gu...,1


In [108]:
df_ratings_with_book.title.unique().shape

(156592,)

In [110]:
df_num_rating = df_num_rating.rename(columns={
    "rating": "num_rating"
})

In [111]:
df_num_rating.columns

Index(['title', 'num_rating'], dtype='object')

In [114]:
print(df_num_rating.shape)
print(df_ratings_with_book.shape)

(156592, 2)
(542394, 8)


In [ ]:
### Get the 